Importing the required libraries

In [1]:
import math
import time
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

from sklearn import metrics
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import LabelEncoder

%matplotlib inline

import the data

In [68]:
heroes_info = pd.read_csv('./superhero-set//heroes_information.csv',index_col = 0)
heroes_powers = pd.read_csv('./superhero-set/super_hero_powers.csv')

In [69]:
heroes_info.head()

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0


In [70]:
heroes_info = heroes_info[heroes_info.Race != "-"]  # ignoring the super heroes with unknown race

In [71]:
heroes_info['Human'] = heroes_info['Race'].apply(lambda i: 1 if i == 'Human' else 0) # creating the label

# Data Preprocessing

In [72]:
heroes_info.columns[heroes_info.isnull().any()]  # checking for null values

Index(['Publisher', 'Weight'], dtype='object')

In [73]:
def null_count(df,col):
    '''
    checking for unnecessary values in the dataset like -, -99.0
    df : dataframe
    col : column
    '''
    null_symbols = ["-",-99.0]
    per = 0
    for i in null_symbols:
        if i in df[col].value_counts():
            if df[col].isnull().sum() > 0:  # checking for null value
                per = ((df[col].value_counts()[i] + df[col].isnull().sum() ) / len(df)) *100
                break
            else:
                per = (df[col].value_counts()[i] / len(df)) *100
                break
        else:
            continue
    return per

In [34]:
cols = ['Gender', 'Eye color', 'Hair color',
       'Height', 'Publisher', 'Skin color', 'Alignment', 'Weight']

In [35]:
removable_columns = []
for i in cols:
    if null_count(heroes_info,i) > 50:  # Removing the columns that have more than 50% of junk records 
        removable_columns.append(i)

In [36]:
removable_columns

['Skin color']

In [37]:
heroes_info.drop(['Skin color','Race'],axis=1,inplace=True)  # droping the skin colour and race columns

In [38]:
heroes_info.describe(include=["O"])   # Descriptive stats of categorical features

,name,Gender,Eye color,Hair color,Publisher,Alignment
count,430,430,430,430,430,430
unique,420,3,21,22,19,4
top,Spider-Man,Male,blue,Black,Marvel Comics,good
freq,3,321,147,106,223,279


In [39]:
heroes_info.describe()  # Descriptive stats of continous features

,Height,Weight,Human
count,430.000000,430.000000,430.000000
mean,188.712791,109.155814,0.483721
std,62.627992,94.114894,0.500317
min,15.200000,4.000000,0.000000
25%,178.000000,74.000000,0.000000
50%,183.000000,83.000000,0.000000
75%,188.000000,99.000000,1.000000
max,975.000000,855.000000,1.000000


In [40]:
heroes_info['Weight'].fillna(heroes_info['Weight'].median(), inplace=True) # Filling null values with median
heroes_info['Publisher'].fillna(heroes_info['Publisher'].mode()[0], inplace=True) # Filling null values with median

In [41]:
from sklearn.preprocessing import Imputer      # Imüuting the missing values

imp = Imputer(missing_values=-99.0, strategy='median', axis=0)
heroes_info["Height"]=imp.fit_transform(heroes_info[["Height"]])
heroes_info["Weight"]=imp.fit_transform(heroes_info[["Weight"]])

Exploratory data analysis

In [ ]:
def plot_distribution(dataset, cols=5, width=20, height=15, hspace=0.2, wspace=0.5):
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure(figsize=(width,height))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=wspace, hspace=hspace)
    rows = math.ceil(float(dataset.shape[1]) / cols)
    for i, column in enumerate(dataset.columns):
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.set_title(column)
        if dataset.dtypes[column] == np.object:
            g = sns.countplot(y=column, data=dataset)
            substrings = [s.get_text()[:18] for s in g.get_yticklabels()]
            g.set(yticklabels=substrings)
            plt.xticks(rotation=25)
        else:
            g = sns.distplot(dataset[column])
            plt.xticks(rotation=25)
    
plot_distribution(heroes_info, cols=3, width=20, height=20, hspace=0.45, wspace=0.5)

From the above distributions we can understand that for few of the features there are too many categorical values
and when we create dummy variables from these it would create too many dimensions. To avoid that we can generalize 
the dimensions by grouping similar items into one category. For instance in Publisher feature we can reduce the dimensions into 3 broad categories (Marvel, DC and Others).

In [ ]:
heroes_powers = heroes_powers * 1    # Converting True , False in the dataframe features to numericals(1 and 0)

In [ ]:
dataset = pd.merge(heroes_info, heroes_powers, left_on=['name'], right_on=['hero_names'], how='inner') # Joining the two datasets

In [ ]:
from sklearn.preprocessing import LabelEncoder   # Encoding the categorical columns
cols = ['Gender', 'Eye color', 'Hair color', 'Publisher', 'Alignment']

dataset[cols] = dataset[cols].apply(LabelEncoder().fit_transform)

In [ ]:
X_columns_drop = ['name', 'hero_names', 'Human']     # Dropping the features which are note required for development of model and the target label
X, y = dataset.drop(X_columns_drop, axis=1), dataset['Human']

In [ ]:
from sklearn.model_selection import train_test_split  # Splitting the dataset for testing purposes
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42,stratify=y)

# Model Building

In [3]:
# Fit the ml algorithm on the datasets
def fit_algo(algo, X_train, y_train, X_test, cv):
    model = algo.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)[:,1]

    acc = round(model.score(X_test, y_test) * 100, 2) 
    # CV 
    train_pred = model_selection.cross_val_predict(algo, 
                                                  X_train, 
                                                  y_train, 
                                                  cv=cv, 
                                                  n_jobs = -1)
    acc_cv = round(metrics.accuracy_score(y_train, train_pred) * 100, 2)
    
    return model, train_pred, test_pred, acc, acc_cv, probs

# calculate the auc value
def plot_roc_curve(y_test, preds):
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
#  DecisionTreeClassifier
start_time = time.time()
model_dt, train_pred_dt, test_pred_dt, acc_dt, acc_cv_dt, probs_dt = fit_algo(DecisionTreeClassifier(), 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             10)
dt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_dt)
print("Accuracy CV 10-Fold: %s" % acc_cv_dt)
print("Running Time: %s" % datetime.timedelta(seconds=dt_time))

In [ ]:
#  RandomForestClassifier
start_time = time.time()
model_rf, train_pred_rf, test_pred_rf, acc_rf, acc_cv_rf, probs_rf = fit_algo(RandomForestClassifier(n_estimators=100), 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             10)
dt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_rf)
print("Accuracy CV 10-Fold: %s" % acc_cv_rf)
print("Running Time: %s" % datetime.timedelta(seconds=dt_time))

In [ ]:
print('Train data :\n', metrics.classification_report(y_train, train_pred_rf))
print('Test data : \n', metrics.classification_report(y_test, test_pred_rf))
plot_roc_curve(y_test, probs_rf)

In [ ]:
#Naive Bayes
start_time = time.time()
model_nb, train_pred_nb, test_pred_nb, acc_nb, acc_cv_nb, probs_nb = fit_algo(GaussianNB(), 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             10)
dt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_nb)
print("Accuracy CV 10-Fold: %s" % acc_cv_nb)
print("Running Time: %s" % datetime.timedelta(seconds=dt_time))

In [ ]:
#GradientBoostingClassifier
start_time = time.time()
model_gb, train_pred_gb, test_pred_gb, acc_gb, acc_cv_gb, probs_gb = fit_algo(GradientBoostingClassifier(n_estimators=100), 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             10)
dt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_gb)
print("Accuracy CV 10-Fold: %s" % acc_cv_gb)
print("Running Time: %s" % datetime.timedelta(seconds=dt_time))

In [ ]:
models = pd.DataFrame({
    'Model': [ 
              'Random Forest', 'Naive Bayes', 
              'Decision Tree', 'Gradient Boosting Trees'],
    'Score': [ 
        acc_rf, 
        acc_nb,  
        acc_dt,
        acc_gb
    ]})
models.sort_values(by='Score', ascending=False)